<a href="https://colab.research.google.com/github/miladhe/Life-Expectancy-Analysis/blob/main/Life_expectancy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os,datetime
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,Lasso,Ridge,SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import io

In [6]:
from google.colab import files
uploaded= files.upload()

Saving Life Expectancy Data.csv to Life Expectancy Data.csv


In [8]:
data=pd.read_csv(io.BytesIO(uploaded["Life Expectancy Data.csv"]))
data.head(5)

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [9]:
data.shape

(2938, 22)

In [10]:
data.isnull().sum()

,0
Country,0
Year,0
Status,0
Life expectancy,10
Adult Mortality,10
infant deaths,0
Alcohol,194
percentage expenditure,0
Hepatitis B,553
Measles,0


In [11]:
countries=data["Country"].unique()
nan_cols=["Life expectancy ","Adult Mortality","Alcohol","Hepatitis B"," BMI ","Polio","Total expenditure","Diphtheria ","GDP"," thinness  1-19 years"," thinness 5-9 years","Population","Income composition of resources"]

for cols in nan_cols:
  for country in countries:
    data.loc[data["Country"]==country,cols]=data.loc[data["Country"]==country,cols].fillna(data[data["Country"]==country][cols].mean())

In [12]:
data.isna().sum()

,0
Country,0
Year,0
Status,0
Life expectancy,10
Adult Mortality,10
infant deaths,0
Alcohol,17
percentage expenditure,0
Hepatitis B,144
Measles,0


In [13]:
data=data.dropna()
data.shape

(2128, 22)

In [14]:
data["Status"].value_counts()

,count
Status,
Developing,1824
Developed,304


In [15]:
data["Country"].value_counts()

,count
Country,
Afghanistan,16
Albania,16
Algeria,16
Angola,16
Argentina,16
...,...
Uruguay,16
Uzbekistan,16
Vanuatu,16


In [16]:
data_corr=data[["Life expectancy ",
                "Adult Mortality",
                "Schooling",
                "Total expenditure",
                "Diphtheria ",
                "GDP",
                "Population"]].corr()

In [17]:
data_corr

,Life expectancy,Adult Mortality,Schooling,Total expenditure,Diphtheria,GDP,Population
Life expectancy,1.000000,-0.662604,0.746619,0.203689,0.449975,0.445425,-0.011420
Adult Mortality,-0.662604,1.000000,-0.408661,-0.097938,-0.212157,-0.259500,-0.021050
Schooling,0.746619,-0.408661,1.000000,0.261679,0.432891,0.471767,-0.022453
Total expenditure,0.203689,-0.097938,0.261679,1.000000,0.183238,0.212498,-0.079752
Diphtheria,0.449975,-0.212157,0.432891,0.183238,1.000000,0.190957,-0.024167
GDP,0.445425,-0.259500,0.471767,0.212498,0.190957,1.000000,-0.016800
Population,-0.011420,-0.021050,-0.022453,-0.079752,-0.024167,-0.016800,1.000000


In [18]:
features=data.drop("Life expectancy ",axis=1)
target=data[["Life expectancy "]]

In [19]:
features.columns

Index(['Country', 'Year', 'Status', 'Adult Mortality', 'infant deaths',
       'Alcohol', 'percentage expenditure', 'Hepatitis B', 'Measles ', ' BMI ',
       'under-five deaths ', 'Polio', 'Total expenditure', 'Diphtheria ',
       ' HIV/AIDS', 'GDP', 'Population', ' thinness  1-19 years',
       ' thinness 5-9 years', 'Income composition of resources', 'Schooling'],
      dtype='object')

In [20]:
target.columns

Index(['Life expectancy '], dtype='object')

In [21]:
target.sample(5)

,Life expectancy
1578,48.5
827,75.0
2263,57.9
1682,71.0
2703,65.6


In [22]:
features=features.drop("Country",axis=1)

In [23]:
features.columns

Index(['Year', 'Status', 'Adult Mortality', 'infant deaths', 'Alcohol',
       'percentage expenditure', 'Hepatitis B', 'Measles ', ' BMI ',
       'under-five deaths ', 'Polio', 'Total expenditure', 'Diphtheria ',
       ' HIV/AIDS', 'GDP', 'Population', ' thinness  1-19 years',
       ' thinness 5-9 years', 'Income composition of resources', 'Schooling'],
      dtype='object')

In [24]:
categorical_features=features["Status"].copy()

categorical_features.head()

,Status
0,Developing
1,Developing
2,Developing
3,Developing
4,Developing


In [25]:
categorical_features=pd.get_dummies(categorical_features)


categorical_features.head()

,Developed,Developing
0,False,True
1,False,True
2,False,True
3,False,True
4,False,True


In [26]:
categorical_features1=categorical_features.replace({True:1,False:0})
categorical_features1.head(5)

/tmp/ipython-input-4018274370.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  categorical_features1=categorical_features.replace({True:1,False:0})


,Developed,Developing
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [27]:
numeric_features=features.drop(["Status"],axis=1)
numeric_features.head()

,Year,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,2015,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,2014,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,2013,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,2012,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,2011,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [28]:
numeric_features.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,2128.0,2.007500e+03,4.610856e+00,2000.00000,2003.750000,2.007500e+03,2.011250e+03,2.015000e+03
Adult Mortality,2128.0,1.743003e+02,1.297593e+02,1.00000,76.000000,1.520000e+02,2.412500e+02,7.230000e+02
infant deaths,2128.0,3.590273e+01,1.362247e+02,0.00000,1.000000,4.000000e+00,2.400000e+01,1.800000e+03
Alcohol,2128.0,4.436893e+00,3.962858e+00,0.01000,0.827500,3.735000e+00,7.162500e+00,1.787000e+01
percentage expenditure,2128.0,6.400720e+02,1.710799e+03,0.00000,19.883256,8.367799e+01,4.313651e+02,1.896135e+04
Hepatitis B,2128.0,7.727206e+01,2.526004e+01,2.00000,68.000000,8.700000e+01,9.500000e+01,9.900000e+01
Measles,2128.0,2.657467e+03,1.191224e+04,0.00000,0.000000,2.000000e+01,4.592500e+02,2.121830e+05
BMI,2128.0,3.669196e+01,1.984373e+01,1.40000,18.400000,3.885000e+01,5.520000e+01,7.760000e+01
under-five deaths,2128.0,4.981720e+01,1.851527e+02,0.00000,1.000000,4.000000e+00,3.425000e+01,2.500000e+03
Polio,2128.0,8.057201e+01,2.417005e+01,3.00000,75.000000,9.100000e+01,9.600000e+01,9.900000e+01


In [29]:
standardscaler=StandardScaler()

numeric_features=pd.DataFrame(standardscaler.fit_transform(numeric_features),
                             columns=numeric_features.columns,
                             index=numeric_features.index)
numeric_features.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,2128.0,0.000000e+00,1.000235,-1.626978,-0.813489,0.000000,0.813489,1.626978
Adult Mortality,2128.0,-5.342427e-17,1.000235,-1.335866,-0.757737,-0.171899,0.516075,4.229591
infant deaths,2128.0,0.000000e+00,1.000235,-0.263617,-0.256275,-0.234247,-0.087396,12.952948
Alcohol,2128.0,1.402387e-16,1.000235,-1.117358,-0.911020,-0.177159,0.687950,3.390549
percentage expenditure,2128.0,1.335607e-17,1.000235,-0.374224,-0.362599,-0.325301,-0.122022,10.711711
Hepatitis B,2128.0,3.005115e-17,1.000235,-2.980588,-0.367151,0.385202,0.701983,0.860373
Measles,2128.0,0.000000e+00,1.000235,-0.223140,-0.223140,-0.221460,-0.184578,17.593236
BMI,2128.0,4.674623e-17,1.000235,-1.778912,-0.922017,0.108777,0.932909,2.061994
under-five deaths,2128.0,-1.669508e-17,1.000235,-0.269123,-0.263721,-0.247514,-0.084097,13.236418
Polio,2128.0,8.347542e-17,1.000235,-3.210181,-0.230588,0.431544,0.638460,0.762610


In [30]:
processed_features=pd.concat([numeric_features,categorical_features1],axis=1,sort=False)
processed_features.head()

,Year,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,...,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Developed,Developing
0,1.626978,0.683732,0.191620,-1.117358,-0.332550,-0.485943,-0.126242,-0.886733,0.179261,-3.086031,...,-0.649601,-0.355934,-0.424250,0.322064,2.566866,2.523280,-0.631802,-0.497055,0,1
1,1.410048,0.745399,0.206305,-1.117358,-0.331238,-0.604736,-0.181828,-0.911936,0.195467,-0.934103,...,-0.773729,-0.355934,-0.421749,-0.207042,2.630549,2.564910,-0.646555,-0.528280,0,1
2,1.193118,0.722274,0.220990,-1.117358,-0.331416,-0.525541,-0.187034,-0.937139,0.211674,-0.768570,...,-0.690977,-0.355934,-0.420074,0.290313,2.673004,2.606539,-0.676060,-0.559506,0,1
3,0.976187,0.753107,0.243018,-1.117358,-0.328513,-0.406748,0.010876,-0.962342,0.233283,-0.561654,...,-0.566848,-0.355934,-0.416713,-0.153680,2.715459,2.668984,-0.710482,-0.590731,0,1
4,0.759257,0.776233,0.257703,-1.117358,-0.370075,-0.367151,0.029853,-0.982504,0.254892,-0.520270,...,-0.525472,-0.355934,-0.470041,-0.165057,2.779142,2.710614,-0.754739,-0.684408,0,1


In [31]:
processed_features.shape

(2128, 21)

In [32]:
x_train,x_test,y_train,y_test=train_test_split(processed_features,target,test_size=0.2,random_state=1)

In [33]:
def linear_reg(x_train,y_train):
  print("Linear_regression")
  lin_reg=LinearRegression()
  lin_reg.fit(x_train,y_train)
  return lin_reg

In [34]:
def lasso_reg(x_train,y_train):
  print("Lasso_Regression")
  las_res=Lasso(alpha=0.8,max_iter=10000)
  las_res.fit(x_train,y_train)
  return las_res

In [35]:
def ridge_reg(x_train,y_train):
  print("Ridg_Regression")
  rige_reg=Ridge(alpha=0.8)
  rige_reg.fit(x_train,y_train)
  return rige_reg

In [36]:
def sgd(x_train,y_train):
  print("Stochastic_Gradient_Descent")
  s=SGDRegressor(max_iter=2000)
  s.fit(x_train,y_train)
  return s

In [37]:
def mlp(x_train,y_train):
  print("Multilayer preceptron Regressor")
  m=MLPRegressor(hidden_layer_sizes={30,30,30},activation="relu")
  m.fit(x_train,y_train)
  return m

In [38]:
def build_and_train_model(data,target_name,reg_fn):
  model=reg_fn(x_train,y_train)
  score=model.score(x_train,y_train)
  print("Training score: ",score)

  y_pred=model.predict(x_test)
  r_score=r2_score(y_test,y_pred)
  print("Testing Score: ",r_score)

  pred_results=pd.DataFrame({"y_test":y_test.values.flatten(),"y_pred ":y_pred.flatten()},index=range(len(y_pred)))
  return (pred_results.sample(10))
  plt.figure(figsize=(10,8))
  plt.plot(y_pred,label="PREDICTED")
  plt.plot(y_test.values,label="Actual")
  plt.ylabel("Life Expectancy")
  plt.legend()
  plt.show()

  return {"model":model,
          "x_train":x_train,"x_test":x_test,
          "y_train":y_train,"y_test":y_test,
          "y_pred":y_pred}

In [39]:
logdir=os.path.join("slogs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard=keras.callbacks.TensorBoard(logdir,histogram_freq=1)

In [40]:
def building_sigmoid_single_layer_model():
  print("Deep Learning Sigmoid Neural Network:")
  model_sig=keras.Sequential([layers.Dense(32,input_shape=(x_train.shape[1],),activation="sigmoid"),layers.Dense(1)])
  model_sig.compile(loss="mse",metrics=["mae","mse"],optimizer=optimizers.Adam(learning_rate=0.01))
  model_sig.summary()
  return model_sig
model_sig=building_sigmoid_single_layer_model()

Deep Learning Sigmoid Neural Network:


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 737 (2.88 KB)

 Trainable params: 737 (2.88 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
train_hist=model_sig.fit(x_train,y_train,epochs=100,validation_split=0.2,verbose=1,callbacks=[tensorboard])

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 4422.4272 - mae: 65.7536 - mse: 4422.4272 - val_loss: 3534.6917 - val_mae: 58.8279 - val_mse: 3534.6917
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 3221.6431 - mae: 56.0142 - mse: 3221.6431 - val_loss: 2191.7085 - val_mae: 45.9141 - val_mse: 2191.7085
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1833.8357 - mae: 41.7130 - mse: 1833.8357 - val_loss: 1068.6274 - val_mae: 31.6260 - val_mse: 1068.6274
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 874.0438 - mae: 28.2420 - mse: 874.0438 - val_loss: 448.9210 - val_mae: 19.6973 - val_mse: 448.9210
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 366.7604 - mae: 17.4260 - mse: 366.7604 - val_loss: 184.3666 - val_mae: 11.8415 - val_mse: 184.3666
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 144.4766 - mae: 10.2776 - mse: 144.4766 - val_loss: 86.9936 - val_mae: 7.8010 - val_mse: 86.9936
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━

In [42]:
%load_ext tensorboard


In [ ]:
%tensorboard --logdir slogs --port 8000 --host 127.0.0.1

Launching TensorBoard...

In [ ]:
logdir=os.path.join("rlogs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard=keras.callbacks.TensorBoard(logdir,histogram_freq=1)

In [ ]:
def building_relu_multiple_layer_model():
  print("Deep Learning model with Relu activation:")
  model_relu=keras.Sequential([layers.Dense(32,input_shape=(x_train.shape[1],),activation="relu"),
                              layers.Dense(16,activation="relu"),
                              layers.Dense(4,activation="relu"),
                              layers.Dense(1)])
  model_relu.compile(loss="mse",metrics=["mae","mse"],optimizer=optimizers.Adam(learning_rate=0.01))
  model_relu.summary()
  return model_relu
model_relu=building_relu_multiple_layer_model()

In [ ]:
training_histr=model_relu.fit(x_train,y_train,epochs=100,validation_split=0.2,batch_size=100,callbacks=[tensorboard])

In [ ]:
logdir=os.path.join("sglogs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard=keras.callbacks.TensorBoard(logdir,histogram_freq=1)

In [ ]:
def building_sgd_multiple_layer_model():
  print("Deep Learning Neural Network Using SGD as Loss Function:")
  model_sgd=keras.Sequential([layers.Dense(32,input_shape=(x_train.shape[1],),activation="relu"),
                              layers.Dense(16,activation="relu"),
                              layers.Dense(4,activation="relu"),
                              layers.Dense(1)])
  model_sgd.compile(loss="mse",metrics=["mae","mse"],optimizer=optimizers.SGD(learning_rate=0.001))
  model_sgd.summary()
  return model_sgd
model_sgd=building_sgd_multiple_layer_model()

In [ ]:
training_histrsgd=model_sgd.fit(x_train,y_train,epochs=100,validation_split=0.2,batch_size=100,callbacks=[tensorboard])

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir sglogs --port 8002 --host 127.0.0.1

In [ ]:
logdir=os.path.join("rmlogs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard=keras.callbacks.TensorBoard(logdir,histogram_freq=1)

In [ ]:
def building_RMprop_multiple_layer_model():
  print("Deep Learning Neural Network Using RMprop as Loss Function:")
  model_rms=keras.Sequential([layers.Dense(16,input_shape=(x_train.shape[1],),activation="elu"),
                              layers.Dense(8,activation="elu"),
                              layers.Dense(4,activation="elu"),
                              layers.Dense(1)])
  model_rms.compile(loss="mse",metrics=["mae","mse"],optimizer=optimizers.RMSprop(learning_rate=0.01))
  model_rms.summary()
  return model_rms
model_rms=building_RMprop_multiple_layer_model()

In [ ]:
training_histrrms=model_rms.fit(x_train,y_train,epochs=100,validation_split=0.2,batch_size=100,callbacks=[tensorboard])

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir rmlogs --port 8003 --host 127.0.0.1

In [ ]:
linear_reg_e=build_and_train_model(data,target,linear_reg)

In [ ]:
lasso_reg_e=build_and_train_model(data,target,lasso_reg)

In [ ]:
rige_reg_e=build_and_train_model(data,target,ridge_reg)

In [ ]:
sgd_reg_e=build_and_train_model(data,target,sgd)

In [ ]:
mlp_reg_e=build_and_train_model(data,target,mlp)